
1. Download the data from <a href='https://drive.google.com/file/d/15dCNcmKskcFVjs7R0ElQkR61Ex53uJpM/view?usp=sharing'>here</a>

2. Code the model to classify data like below image

<img src='https://i.imgur.com/33ptOFy.png'>

3. Write your own callback function, that has to print the micro F1 score and AUC score after each epoch.

4. Save your model at every epoch if your validation accuracy is improved from previous epoch. 

5. you have to decay learning based on below conditions 
        Cond1. If your validation accuracy at that epoch is less than previous epoch accuracy, you have to decrese the
               learning rate by 10%. 
        Cond2. For every 3rd epoch, decay your learning rate by 5%.
        
6. If you are getting any NaN values(either weigths or loss) while training, you have to terminate your training. 

7. You have to stop the training if your validation accuracy is not increased in last 2 epochs.

8. Use tensorboard for every model and analyse your gradients.

9. Use cross entropy as loss function

10. Try the architecture params as given below. 

<pre>
<b>Model-1</b>
<pre>
1. Use tanh as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-2</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-3</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use he_uniform() as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-4</b>
<pre>
1. Try with any values to get better accuracy/f1 score.  
</pre>
</pre>

In [102]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard

In [3]:
df = pd.read_csv('data.csv')

In [4]:
df.shape

(20000, 3)

In [5]:
df.head()

,f1,f2,label
0,0.450564,1.074305,0.0
1,0.085632,0.967682,0.0
2,0.117326,0.971521,1.0
3,0.982179,-0.380408,0.0
4,-0.720352,0.955850,0.0


In [8]:
X = df.drop(['label'], axis=1)
y = df['label'].values

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((13400, 2), (6600, 2), (13400,), (6600,))

In [17]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((10720, 2), (2680, 2), (10720,), (2680,))

#### Callback to get micro F1 score and AUC score after each epoch

In [62]:
from sklearn.metrics import  f1_score, roc_auc_score
class PrintMicroF1andAUCCallback(tf.keras.callbacks.Callback):

    def __init__(self, validation_data):
        super().__init__()
        self.micro_f1_score = tf.keras.metrics.F1Score(name="micro_f1_score")
        self.auc_score = tf.keras.metrics.AUC(name='auc_score')
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs):
        x_val, y_val = self.validation_data
        y_pred = (np.asarray(self.model.predict(x_val))).round()        
        
        micro_f1_score = f1_score(y_val, y_pred, average='micro')
        auc_score = roc_auc_score(y_val, y_pred)
        
        print(f"Epoch: {epoch}, Micro F1 Score: {micro_f1_score}, AUC Score: {auc_score}")

#### Learning rate decay callback

In [92]:
class CustomLearningRateScheduler(keras.callbacks.Callback):
    """Learning rate scheduler which sets the learning rate according to schedule.

    Arguments:
        schedule: a function that takes an epoch index
            (integer, indexed from 0) and current learning rate
            as inputs and returns a new learning rate as output (float).
    """

    def __init__(self, schedule):
        super().__init__()
        self.schedule = schedule

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('Optimizer must have a "lr" attribute.')
        # Get the current learning rate from model's optimizer.
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        # Call schedule function to get the scheduled learning rate.
        scheduled_lr = self.schedule(epoch, lr)
        # Set the value back to the optimizer before this epoch starts
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        print("\nEpoch %05d: Learning rate is %6.4f." % (epoch, scheduled_lr))

def lr_schedule(epoch, lr):
    """Helper function to retrieve the scheduled learning rate based on epoch."""
    if epoch > 0 and epoch%3 == 0:
        return lr - (lr*5/100)
    return lr

#### Callback to Terminate on NaN values in Weights or Loss

In [95]:
class TerminateOnNaNCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        for weight in self.model.weights:
            if tf.reduce_any(tf.math.is_nan(weight)):
                print(f"Epoch {epoch + 1}: NaN values detected in model weights. Training terminated.")
                self.model.stop_training = True
        
        if tf.math.is_nan(logs['loss']):
            print(f"Epoch {epoch + 1}: NaN values detected in loss. Training terminated.")
            self.model.stop_training = True

#### Callback to Terminate training on no improvement in Val accuracy

In [98]:
class EarlyStoppingOnValidationAccuracy(tf.keras.callbacks.Callback):
    def __init__(self, patience=2):
        super().__init__()
        self.patience = patience
        self.wait = 0
        self.best_val_acc = float('-inf')

    def on_epoch_end(self, epoch, logs=None):
        val_acc = logs.get('val_accuracy')
        
        if val_acc is None:
            raise ValueError("Validation accuracy is not found in logs. Make sure you are using 'val_accuracy' as the validation metric.")
        
        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                print(f"Epoch {epoch + 1}: Validation accuracy has not improved for {self.patience} consecutive epochs. Training terminated.")
                self.model.stop_training = True

In [106]:
# Define the Keras model to add callbacks to
def get_model(activation, initializer):
    #Input layer
    input_layer = Input(shape=(2,))

    #Dense hidden layer
    layer1 = Dense(50,activation=activation,kernel_initializer=initializer)(input_layer)
    layer2 = Dense(40,activation=activation,kernel_initializer=initializer)(layer1)
    layer3 = Dense(30,activation=activation,kernel_initializer=initializer)(layer2)
    layer4 = Dense(20,activation=activation,kernel_initializer=initializer)(layer3)
    layer5 = Dense(10,activation=activation,kernel_initializer=initializer)(layer4)

    #output layer
    output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.RandomUniform(0, 1))(layer5)

    #Creating a model
    model = Model(inputs=input_layer,outputs=output)
    return model

In [107]:

#Callbacks
f1_auc_callback = PrintMicroF1andAUCCallback(
                    validation_data=(X_val, y_val))
checkpoint_filepath = "model_save"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                        filepath=checkpoint_filepath,
                        save_weights_only=True,
                        monitor='val_accuracy',
                        mode='max',
                        save_best_only=True)
reduce_lr_callback = CustomLearningRateScheduler(lr_schedule)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_accuracy', 
                factor=0.1,
                patience=1, 
                min_lr=1e-7)
terminate_nan_callback = TerminateOnNaNCallback()
val_acc_terminate_callback = EarlyStoppingOnValidationAccuracy(patience=2)

log_dir = "logs/gradient"  # Specify the log directory
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)

callbacks = [f1_auc_callback, 
                checkpoint_callback,
                reduce_lr_callback,
                reduce_lr,
                terminate_nan_callback,
                val_acc_terminate_callback,
                tensorboard_callback]

### Model 1

In [113]:
optimizer = tf.keras.optimizers.SGD(0.1, 0.9)
initializer = tf.keras.initializers.RandomUniform(0, 1, seed=100)
model = get_model('tanh', initializer)
# model.summary()

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=16, callbacks=callbacks)


Epoch 00000: Learning rate is 0.1000.
Epoch 1/10
84/84 [==============================] - 0s 984us/steposs: 0.7373 - accuracy: 0.
Epoch: 0, Micro F1 Score: 0.5406716417910448, AUC Score: 0.5339568741292128
670/670 [==============================] - 3s 3ms/step - loss: 0.7368 - accuracy: 0.5206 - val_loss: 0.7923 - val_accuracy: 0.5407 - lr: 0.1000

Epoch 00001: Learning rate is 0.1000.
Epoch 2/10
84/84 [==============================] - 0s 959us/steposs: 0.7360 - accuracy: 0.
Epoch: 1, Micro F1 Score: 0.5406716417910448, AUC Score: 0.5339568741292128
670/670 [==============================] - 2s 2ms/step - loss: 0.7417 - accuracy: 0.5098 - val_loss: 0.7268 - val_accuracy: 0.5407 - lr: 0.1000

Epoch 00002: Learning rate is 0.0100.
Epoch 3/10
84/84 [==============================] - 0s 1ms/step loss: 0.6875 - accuracy: 0.
Epoch: 2, Micro F1 Score: 0.5406716417910448, AUC Score: 0.5339568741292128
Epoch 3: Validation accuracy has not improved for 2 consecutive epochs. Training terminated

### Model 2

In [116]:
optimizer = tf.keras.optimizers.SGD(0.1, 0.9)
initializer = tf.keras.initializers.RandomUniform(0, 1, seed=50)
model = get_model('relu', initializer)
# model.summary()

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=16, callbacks=callbacks)


Epoch 00000: Learning rate is 0.1000.
Epoch 1/10
84/84 [==============================] - 0s 1ms/step loss: 109.0597 - accuracy: 
Epoch: 0, Micro F1 Score: 0.4914179104477612, AUC Score: 0.5
Epoch 1: Validation accuracy has not improved for 2 consecutive epochs. Training terminated.
670/670 [==============================] - 3s 3ms/step - loss: 106.4719 - accuracy: 0.4983 - val_loss: 0.6933 - val_accuracy: 0.4914 - lr: 0.1000


### Model 3

In [119]:
optimizer = tf.keras.optimizers.SGD(0.1, 0.9)
initializer = tf.keras.initializers.he_uniform(seed=50)
model = get_model('relu', initializer)
# model.summary()

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=16, callbacks=callbacks)


Epoch 00000: Learning rate is 0.1000.
Epoch 1/10
84/84 [==============================] - 0s 965us/steposs: 0.6944 - accuracy: 0.
Epoch: 0, Micro F1 Score: 0.5085820895522388, AUC Score: 0.5
Epoch 1: Validation accuracy has not improved for 2 consecutive epochs. Training terminated.
670/670 [==============================] - 3s 3ms/step - loss: 0.6944 - accuracy: 0.5183 - val_loss: 0.7126 - val_accuracy: 0.5086 - lr: 0.1000
